In [3]:
import os
import netCDF4
import numpy as np
import matplotlib.pyplot as plt

In [23]:
gp = 2952799
path_to_nc_file = "../../../Download_SWI/c_gls_SWI1km_202107011200_CEURO_SCATSAR_V1.0.1.nc"

In [ ]:
# Get point grid (pg) identifiers from https://dgg.geo.tuwien.ac.at/
pg_finland = "/Users/casado/projects/CASSINI/Download_SWI/pointlist_Finland_copernicus_swi_ts.csv"

In [28]:
with netCDF4.Dataset(path_to_nc_file) as ncdata:
    #loc_idx = np.where(ncdata.variables['location_id'][:] == gp)[0]
    swi15 = ncdata['SWI_015'].values()
    print(swi15)
    #print(loc_idx)

AttributeError: NetCDF: Attribute not found

In [49]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'satellite-soil-moisture',
    {
        'format': 'zip',
        'variable': 'surface_soil_moisture',
        'time_aggregation': 'day_average',
        'year': [
            '2017', '2018', '2019',
            '2020',
        ],
        'month': [
            '05', '06', '07',
            '08',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'type_of_record': 'cdr',
        'version': 'v202012.0.0',
        'type_of_sensor': 'active',
    },
    'download.zip')

2022-11-05 20:32:05,544 INFO Welcome to the CDS
2022-11-05 20:32:05,545 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/satellite-soil-moisture
2022-11-05 20:32:05,617 INFO Request is queued
2022-11-05 20:32:06,673 INFO Request is running
2022-11-05 20:38:24,241 INFO Request is completed
2022-11-05 20:38:24,241 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data0/dataset-satellite-soil-moisture-f8a37485-3775-4c81-9601-eec205e7c2d9.zip to download.zip (817.4M)
2022-11-05 20:57:49,399 INFO Download rate 718.4K/s


Result(content_length=857119050,content_type=application/zip,location=https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data0/dataset-satellite-soil-moisture-f8a37485-3775-4c81-9601-eec205e7c2d9.zip)

In [30]:
filepath="download/C3S-SOILMOISTURE-L3S-SSMV-PASSIVE-DAILY-20220926000000-ICDR-v202012.0.1.nc"
nc = netCDF4.Dataset(filepath)

In [48]:
nc.variables['sm'][60:30]



masked_array(
  data=[],
  mask=False,
  fill_value=1e+20,
  dtype=float32)